In [ ]:
# https://www.testpreptraining.com/tutorial/exam-dp-100-designing-implementing-a-data-science-solution-on-azure/
# https://www.gratisexam.com/microsoft/dp-100-exam/
# https://medium.com/deep-ai/study-guide-for-microsoft-azure-data-scientist-associate-certification-dp-100-c2e4611cb071

# https://docs.microsoft.com/en-us/azure/devops/?view=azure-devops&viewFallbackFrom=vsts

In [2]:
# !pip install azureml-core


In [3]:
# !pip install azure-core

In [1]:
import sys

In [2]:
# sys.path

In [25]:
# Importing all the libraries
import azure.core
from azureml.core import Workspace,Environment,Experiment,ScriptRunConfig
from azureml.core.compute import AmlCompute,ComputeTarget
from azureml.core import Run

In [26]:
try:
    ws = Workspace.from_config()
except:
    ws = Workspace.create('MLlearnv1',resource_group='AzureMLV',
                     subscription_id= '76002742-0a9e-44d5-9d98-143f3a2bf30f',
                     create_resource_group=False,
                     location ='Southeast Asia')
    ws.write_config('.azureml')
    

In [27]:
# To get the details
ws.from_config()

Workspace.create(name='MLlearnv1', subscription_id='76002742-0a9e-44d5-9d98-143f3a2bf30f', resource_group='AzureMLV')

In [28]:
# Creating Computer Cluster
try:
    cluster = ComputeTarget(ws,'MLlearn-cluster')
except:
    config= AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                 min_nodes=0,
                                                 max_nodes=4,
                                                 idle_seconds_before_scaledown=2400)
    cluster=ComputeTarget.create(ws,'MLlearn-cluster',config)

In [29]:
ws.compute_targets

{'MLlearn-cluster': AmlCompute(workspace=Workspace.create(name='MLlearnv1', subscription_id='76002742-0a9e-44d5-9d98-143f3a2bf30f', resource_group='AzureMLV'), name=MLlearn-cluster, id=/subscriptions/76002742-0a9e-44d5-9d98-143f3a2bf30f/resourceGroups/AzureMLV/providers/Microsoft.MachineLearningServices/workspaces/MLlearnv1/computes/MLlearn-cluster, type=AmlCompute, provisioning_state=Succeeded, location=southeastasia, tags={})}

In [15]:
# ws.datastores

# Creating Experiment

In [30]:
exp = Experiment(ws,'MLlearnExperiment')

In [7]:
exp

Name,Workspace,Report Page,Docs Page
MLlearnExperiment,MLlearnv1,Link to Azure Machine Learning studio,Link to Documentation


In [31]:
# Executing below code if the evironment and libraries are already available in AzureML , 
# if not then following need to be executed following code 

pythonconfig = ScriptRunConfig(source_directory='./',script='AzurePrac2.py',compute_target='MLlearn-cluster')
execution=exp.submit(pythonconfig)

In [9]:
# Creating Environment in Azure
env = Environment.from_conda_specification(name='virtualenv',file_path='./envfile2.yml')
pythonconfig = ScriptRunConfig(source_directory='./',script='AzurePrac2.py',compute_target='MLlearn-cluster',environment=env)
execution=exp.submit(pythonconfig)

In [23]:
# Passing arguments on runtime ========= First Iteration 
env = Environment.from_conda_specification(name='virtualenv',file_path='./envfile2.yml')
pythonconfig = ScriptRunConfig(source_directory='./',script='AzurePrac2.py',compute_target='MLlearn-cluster',
                               arguments=['--min_samples_leaf',5,'--min_samples_split',8,'--max_depth',120,'--n_estimators',700],
                               environment=env)
execution= exp.submit(pythonconfig)

In [32]:
# Passing arguments on runtime ========= Second Iteration with change in the arugments
env = Environment.from_conda_specification(name='virtualenv',file_path='./envfile2.yml')
pythonconfig = ScriptRunConfig(source_directory='./',
                               script='AzurePrac2.py',
                               compute_target='MLlearn-cluster',
                               arguments=['--min_samples_leaf',5,'--min_samples_split',5,'--max_depth',150,'--n_estimators',1000],
                               environment=env)
execution= exp.submit(pythonconfig)


# Registring the Model in AzureML Studio

In [33]:
from azureml.core import Model

Model.register(ws,model_path='./RegressorModel.pkl',model_name="RegressionModelDectwentyonetwo")

Registering model RegressionModelDectwentyonetwo


Model(workspace=Workspace.create(name='MLlearnv1', subscription_id='76002742-0a9e-44d5-9d98-143f3a2bf30f', resource_group='AzureMLV'), name=RegressionModelDectwentyonetwo, id=RegressionModelDectwentyonetwo:1, version=1, tags={}, properties={})

Deploying the Model  

# Preparing Scoring/prediction Script
# Creating Inference Environment
# Define Inference Config
# Choosing the compute Target
# Deploty Model

In [34]:
from azureml.core.model import InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies

In [35]:
predEnv = CondaDependencies()

In [36]:
predEnv.add_conda_package('scikit-learn')
predEnv.add_pip_package('azure-sdk')

In [37]:
predEnv

# Saving all the required environment details in yml file 

<!-- https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice(class)?view=azure-ml-py -->

<!-- Azure sdk code -->

In [39]:
env_file = './predEnv.yml'

with open(env_file,'w') as f:
    f.write(predEnv.serialize_to_string())
print('Saved dependency info in',env_file)

Saved dependency info in ./predEnv.yml


Inference Config

In [40]:
from azureml.core.model import InferenceConfig

inferconfig = InferenceConfig(runtime='python',source_directory='./',entry_script='ScoreOrPrediction.py',conda_file=env_file)

Chossing Compute Target

In [41]:
from azureml.core.webservice import AciWebservice

ACIconfig = AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=1)

In [42]:
model = Model(ws,'RegressionModelDectwentyonetwo')

<!-- EndpointName is 'ScoreRegressor' -->

In [43]:
deploy_model = Model.deploy(ws,"scoreregressor",
                        models=[model],
                        inference_config=inferconfig,
                        deployment_config=ACIconfig,
                        deployment_target=predEnv,
                        overwrite=True)